In [36]:
import torch
import torchtext 
from torchtext import data, datasets
import pandas as pd


In [37]:
df_phrase = pd.read_csv("dictionary.txt", sep='\|', header=None)
df_label = pd.read_csv("sentiment_labels.txt", sep='|')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [38]:
print(f"Dictionary: {df_phrase.shape}")
print(f"Sentimenet: {df_label.shape}")

Dictionary: (239232, 2)
Sentimenet: (239232, 2)


In [39]:
len(df_phrase), len(df_label)


(239232, 239232)

In [44]:
# Merging Dictionary & Sentiment Label file with ID value column 1 of dictionary & column 0 of sentiment
df = pd.merge(df_phrase , df_label, how='inner', left_on=1, right_on='phrase ids' )
df.shape
df.head()

,0,1,phrase ids,sentiment values
0,!,0,0,0.50000
1,! ',22935,22935,0.52778
2,! '',18235,18235,0.50000
3,! Alas,179257,179257,0.44444
4,! Brilliant,22936,22936,0.86111
